<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [6]:
%bash
gcloud config get-value project

qwiklabs-gcp-0e6fd357ee5ebeda


In [10]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-0e6fd357ee5ebeda'
PROJECT = 'qwiklabs-gcp-0e6fd357ee5ebeda'
REGION = 'eu-west3'

In [11]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [12]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [45]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth,
  MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))), 100) as hashmonth_mod
FROM
  publicdata.samples.natality
WHERE year > 2000
"""
## inspect data from query:
dat_tmp = bq.Query(query + " LIMIT 100").execute().result().to_dataframe()
dat_tmp.head(n = 20)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth,hashmonth_mod
0,8.624484,False,34,1,39,7108882242435606404,4
1,6.437498,False,24,1,40,7108882242435606404,4
2,8.249698,False,21,1,40,7108882242435606404,4
3,7.063611,False,18,1,40,6392072535155213407,7
4,8.190173,False,32,1,38,6392072535155213407,7
5,6.876218,False,22,1,37,6392072535155213407,7
6,5.000084,False,35,2,36,6392072535155213407,7
7,7.251004,False,23,1,39,7146494315947640619,19
8,8.562754,False,18,1,40,7146494315947640619,19
9,6.876218,False,30,1,40,7146494315947640619,19


## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use MOD() on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>

In [17]:
## get size of whole dataset:
query = """
SELECT COUNT(*)
FROM
  publicdata.samples.natality
WHERE year > 2000
"""
bq.Query(query).execute().result().to_dataframe()

f0_
33271914


In [47]:
## calculate target sample size:
target_n_all = 15000
pop_n = 33271914
# print(pop_n / target_n_all)
print(target_n_all / pop_n)

0.0004508306916157574


In [56]:
## try out sampling (size):
query = """
SELECT COUNT(*) 
FROM 
(
  SELECT 
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE year > 2000
  AND MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))), 10) < 8           -- training
  -- AND MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))), 10) IN (8, 9)  -- evaluation
  AND RAND() < 0.00045083
) as tmptable
"""
bq.Query(query).execute().result().to_dataframe()

,f0_
0,11884


In [59]:
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE 
  year > 2000 
  AND MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))), 10) < 8
  AND RAND() < 0.00045083
"""
dat_train = bq.Query(query).execute().result().to_dataframe()
print(dat_train.shape)

(12189, 6)


In [60]:
dat_train.head(n = 5)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.517763,False,12,1,40.0,1866590652208008467
1,1.873929,True,13,1,22.0,3395086207840548940
2,7.125340,False,13,1,39.0,5803731217271494641
3,7.253208,False,14,1,40.0,5934265245228309013
4,8.531890,False,14,1,38.0,1088037545023002395


In [61]:
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE 
  year > 2000 
  AND MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))), 10) IN (8, 9)
  AND RAND() < 0.00045083
"""
dat_eval = bq.Query(query).execute().result().to_dataframe()
print(dat_eval.shape)

(2873, 6)


In [62]:
dat_eval.head(n = 10)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.812284,False,14,1,37.0,4740473290291881219
1,6.448521,True,14,1,47.0,7146494315947640619
2,9.124933,False,14,1,40.0,3572456083892950068
3,7.165024,False,14,1,37.0,4331749811015213748
4,7.063611,False,15,1,42.0,4740473290291881219
5,7.374463,True,15,1,40.0,4740473290291881219
6,5.758474,True,15,1,38.0,6544754656115871769
7,6.188376,False,15,1,36.0,7146494315947640619
8,6.124442,True,15,1,40.0,7146494315947640619
9,6.508046,True,15,1,39.0,5742197815970064689


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hint (highlight to see): <p>
Filtering:
<pre style='color:white'>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre style='color:white'>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre style='color:white'>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [ ]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [ ]:
%bash
wc -l *.csv
head *.csv
tail *.csv

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License